In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import models
import h5py
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import MaxPooling3D
from keras.utils import plot_model
from keras.models import Model

import matplotlib.pyplot as plt
import numpy as np

import os

In [3]:
def denormalize(data,true_min, true_max):
  denormalized = data * (true_max - true_min) + true_min
  return denormalized


def denormalize_full(data, true_minX, true_maxX, true_minY, true_maxY):  #data = (x,y)
  Xgazedenorm = denormalize(data[:,0], true_minX, true_maxX)   #denorm
  Ygazedenorm = denormalize(data[:,1], true_minY, true_maxY)

  Xgazedenorm = np.array(Xgazedenorm)
  Ygazedenorm = np.array(Ygazedenorm)
  final_denormalized = np.column_stack((Xgazedenorm,Ygazedenorm))


  return final_denormalized     #data tis morfis (x,y)


Xmin = -19.067780871643322
Xmax = 21.46873726490827

Ymin = -38.03606803389029
Ymax = 27.631278570342523

In [ ]:
filename = "/content/drive/MyDrive/Thesis/Datasets2/final_dtset/final_dataset2.hdf5"
model = models.load_model("/content/drive/MyDrive/Thesis/Models/Full_10.h5")

low = 26000
high = 26001
with h5py.File(filename, 'r') as f:
    X1_dataset = f['X1_dataset']
    X2_dataset = f['X2_dataset']
    Y_dataset = f['Y_dataset']

    print(X2_dataset.shape)
    test_data1 = X1_dataset[low:high]
    test_data2= X2_dataset[low:high]
    y_data = Y_dataset[low:high]
    predictions = model.predict((test_data1,test_data2))
    print("Prediction:", predictions)
    print("GT:        ", y_data)




    final_predictions = denormalize_full(predictions, Xmin, Xmax, Ymin, Ymax)
    final_groundTruth = denormalize_full(y_data, Xmin, Xmax, Ymin, Ymax)
    print("Prediction:", final_predictions)
    print("GT:        ", final_groundTruth)

    print(test_data1.shape)
    last_image = test_data1[0, -1, :, :, :]

# Check the shape of the extracted image
print(last_image.shape)  # Should output (71, 128, 3)


In [54]:
import random
from time import time
# Load dataset and model
filename = "/content/drive/MyDrive/Thesis/Datasets2/final_dataset.hdf5"
model = models.load_model("/content/drive/MyDrive/Thesis/Models/Full_10.h5")

# Select random samples
num_samples = 44981    # Total sample size
random_indices = random.sample(range(num_samples), 100)  # Select 10 random indices

# Prepare constants
horizontal_fov = 104  # Horizontal FOV in degrees
vertical_fov = 104    # Vertical FOV in degrees
region_radius_angle = 15  # Circle radius in visual degrees
image_width = 128  # Image width
image_height = 71  # Image height

# Helper function to convert visual angles to pixel coordinates
def visual_angle_to_pixel(x_angle, y_angle, image_width, image_height, h_fov, v_fov):
    x_pixel = int((x_angle / h_fov) * image_width + image_width / 2)
    y_pixel = int((-y_angle / v_fov) * image_height + image_height / 2)
    x_pixel = np.clip(x_pixel, 0, image_width - 1)
    y_pixel = np.clip(y_pixel, 0, image_height - 1)
    return x_pixel, y_pixel

# To store the runtime for each prediction
runtimes = []

# Loop over each randomly selected index and plot the results
for idx in random_indices:
    with h5py.File(filename, 'r') as f:
        X1_dataset = f['X1_dataset']
        X2_dataset = f['X2_dataset']
        Y_dataset = f['Y_dataset']

        # Get the corresponding data
        test_data1 = X1_dataset[idx:idx+1]
        test_data2 = X2_dataset[idx:idx+1]
        y_data = Y_dataset[idx:idx+1]

        # Predict the gaze point
        start = time()
        predictions = model.predict((test_data1, test_data2))
        prediction_runtime = (time() - start) * 1000
        runtimes.append(prediction_runtime)


        # Denormalize the predictions and ground truth
        final_predictions = denormalize_full(predictions, Xmin, Xmax, Ymin, Ymax)
        final_groundTruth = denormalize_full(y_data, Xmin, Xmax, Ymin, Ymax)

        # Extract the gaze points and ground truth
        gaze_point_angle = (final_predictions[0, 0], final_predictions[0, 1])
        groundtruth_angle = (final_groundTruth[0, 0], final_groundTruth[0, 1])

        # Get the last image from the sequence
        last_image = test_data1[0, -1, :, :, :]  # Shape (71, 128, 3)
        original_img = (last_image * 255.0).astype(np.uint8)

        # Convert the gaze point and ground truth to pixel coordinates
        gaze_x, gaze_y = visual_angle_to_pixel(*gaze_point_angle, image_width, image_height, horizontal_fov, vertical_fov)
        gt_x, gt_y = visual_angle_to_pixel(*groundtruth_angle, image_width, image_height, horizontal_fov, vertical_fov)

        # Convert the visual angle radius to pixel radius
        horizontal_pixel_per_degree = image_width / horizontal_fov
        vertical_pixel_per_degree = image_height / vertical_fov
        radius_x = region_radius_angle * horizontal_pixel_per_degree
        radius_y = region_radius_angle * vertical_pixel_per_degree
        radius_pixel = int((radius_x + radius_y) / 2)  # Average for a circular region

        # Plot the image for this sample
        fig, ax = plt.subplots()
        ax.imshow(original_img)

        # Plot groundtruth with foveal region
        ax.plot(gt_x, gt_y, marker='+', color='violet', markersize=17, markeredgewidth=3)
        gaze_circle = plt.Circle((gt_x, gt_y), radius_pixel, color='violet', fill=False, linewidth=2)
        ax.add_patch(gaze_circle)

        # Plot gaze point with cross
        ax.plot(gaze_x, gaze_y, marker='+', color='yellow', markersize=17, markeredgewidth=3)
        # Optional: Plot circle for gaze region around gaze point
        # gaze_circle = plt.Circle((gaze_x, gaze_y), radius_pixel, color='red', fill=False, linewidth=1, label="Gaze Region")
        # ax.add_patch(gaze_circle)

        # Plot mean baseline with cross
        mean_bsline = (1.1891425755626226, -5.259580611647258)
        mean_x, mean_y = visual_angle_to_pixel(*mean_bsline, image_width, image_height, horizontal_fov, vertical_fov)
        ax.plot(mean_x, mean_y, marker='+', color='cyan', markersize=17, markeredgewidth=3)

        # Plot center baseline with cross
        center_bsline = (0, 0)
        center_x, center_y = visual_angle_to_pixel(*center_bsline, image_width, image_height, horizontal_fov, vertical_fov)
        ax.plot(center_x, center_y, marker='+', color='red', markersize=17, markeredgewidth=3)

        # Add legend and finalize
        ax.legend(loc="upper left")
        #plt.title(f"Gaze Prediction {idx+1}")
        plt.axis('off')
        plt.show()
        print("Dataset sample: ", idx)


#Calculate average runtime
average_runtime = np.mean(runtimes)
print(f"Average Prediction Runtime: {average_runtime:.4f} ms")
print("FULL RUNTIMES:\n", runtimes)

Output hidden; open in https://colab.research.google.com to view.